<a href="https://colab.research.google.com/github/tema7707/My-little-Kaggle-projects/blob/master/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment analysis (without sentiments)

Download dataset from Kaggle

https://www.kaggle.com/c/iad-deep-learning-sentiment

In [0]:
# download data frome kaggle
!pip install -q kaggle/
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!kaggle competitions download -c iad-deep-learning-sentiment
!mkdir input
!unzip x_test.txt.zip -d input
!unzip x_train.txt.zip -d input
!unzip y_train.csv.zip -d input
!unzip random_prediction.csv.zip -d '/content/drive/My Drive/sentiment analysis'

Import packages and read data using pandas

In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.metrics import make_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

pred=pd.read_csv('/content/drive/My Drive/sentiment analysis/random_prediction.csv')
y_train = pd.read_csv('/input/y_train.csv')
y_train = y_train['Probability']

with open('/input/x_train.txt', 'r') as f:
    x_train = f.readlines()
with open('/input/x_test.txt', 'r') as f:
    x_test = f.readlines()
x_train = pd.DataFrame(x_train, columns=['reviews'])['reviews']
x_test = pd.DataFrame(x_test, columns=['reviews'])['reviews']

In [5]:
vectorizer = TfidfVectorizer(encoding='utf8', 
                             min_df=5, 
                             ngram_range=(1, 2), 
                             max_features=10000)
vectorizer.fit(x_train[0:int(1e6)])

X_train = vectorizer.transform(x_train)
X_test = vectorizer.transform(x_test)
print(X_train.shape, X_test.shape)

(3600000, 10000) (400000, 10000)


In [6]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
preds = lr.predict_proba(X_test)[:,1]

In [27]:
pred=pd.read_csv('/content/drive/My Drive/sentiment analysis/random_prediction.csv')
pred['Probability'] = preds
pred = pred['Probability'].round()
pred = pred.reset_index()
pred.columns = ['Id', 'Probability']
pred['Id'] = pred['Id'] + 1
print(pred)
pred.to_csv('/content/drive/My Drive/sentiment analysis/results.csv',index=False)

            Id  Probability
0            1          1.0
1            2          1.0
2            3          0.0
3            4          0.0
4            5          1.0
...        ...          ...
399995  399996          0.0
399996  399997          0.0
399997  399998          0.0
399998  399999          1.0
399999  400000          0.0

[400000 rows x 2 columns]
